In [24]:
import cv2
import numpy as np
import os

## Initializing haar cascade face detector

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## KNN classifier

In [20]:
def dist(x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))

def knn(X,Y,queryPoint,k=5):
    
    vals = []
    m = X.shape[0]
    
    for i in range(m):
        d = dist(queryPoint,X[i])
        vals.append((d,Y[i]))
        
    vals = sorted(vals)
    
    # Nearest/first k points
    vals = vals[:k]
    
    vals = np.array(vals)
    
    new_vals = np.unique(vals[:,1],return_counts=True)
    
    index = new_vals[1].argmax()
    pred = new_vals[0][index]
    
    return pred

## Function to collect and prepare the training data real time

In [39]:
def train():
    
    cap = cv2.VideoCapture(0)
    skip = 0
    photos_clicked = []
    
    dataset_path = './photos/'
    file_name = input("Enter your name: ")
    
    while True:
        
        skip += 1
        
        ret,frame = cap.read()
        
        if ret == False:
            continue
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(frame,1.3,5)
        faces = sorted(faces,key=lambda f:f[2]*f[3])
    
        # Pick the last face as it will be the largest
        for face in faces[-1:]:
            x,y,w,h = face
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
            # Extract the region of interest
            offset = 10
            face_section = gray_frame[y-offset:y+h+offset,x-offset:x+w+offset]
            face_section = cv2.resize(face_section,(100,100))
        
            if skip%10 == 0:
                photos_clicked.append(face_section)
                print(len(photos_clicked))
    
        cv2.imshow("Frame",frame)
    
        key_pressed = cv2.waitKey(1) & 0xFF
        if key_pressed == ord('q'):
            break
            
    # Convert our face list array to a numpy array
    photos_clicked = np.asarray(photos_clicked)
    photos_clicked = photos_clicked.reshape((photos_clicked.shape[0],-1))
    print(photos_clicked.shape)

    # Save this data to the file system
    np.save(dataset_path+file_name+'.npy',photos_clicked)
    print("Images saved successfully!")
    
    cap.release()
    cv2.destroyAllWindows()

## Function to test the classifier

In [74]:
def test():
    
    cap = cv2.VideoCapture(0)
    
    print("Press 'c' to click a photo")
    
    dataset_path = './photos/'

    face_data = []
    labels = []

    class_id = 0 # Labels for the given file
    names = {} # Mapping between id - name

    # Data preparation

    for fx in os.listdir(dataset_path):
        if fx.endswith('.npy'):
        
            data_item = np.load(dataset_path+fx)
            # print("Loaded "+fx)
            face_data.append(data_item)
        
            # Create labels for the class
            target = class_id*np.ones((data_item.shape[0],))
            labels.append(target)
            names[class_id] = fx[:-4]
            class_id += 1
        
    face_data = np.concatenate(face_data,axis=0)
    labels = np.concatenate(labels,axis=0)
    
    test_point = np.zeros((100,100))
    
    while True:
        
        ret,frame = cap.read()
        
        if ret == False:
            continue
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(frame,1.3,5)
        faces = sorted(faces,key=lambda f:f[2]*f[3])
    
        # Pick the last face as it will be the largest
        for face in faces[-1:]:
            x,y,w,h = face
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
            # Extract the region of interest
            offset = 10
            face_section = gray_frame[y-offset:y+h+offset,x-offset:x+w+offset]
            test_point = cv2.resize(face_section,(100,100))
    
        cv2.imshow("Frame",frame)
    
        key_pressed = cv2.waitKey(1) & 0xFF
        
        if key_pressed == ord('c'):
            y = test_point.reshape((-1,))
            label = knn(face_data,labels,y)
            print("Hello "+names[int(label)])
            break
        
        if key_pressed == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()